In [1]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==custom=py36h363777c_0
  - defaults/win-64::astropy==3.1.2=py36he774522_0
  - defaults/win-64::blaze==0.10.1=py35_0
  - defaults/win-64::bokeh==1.0.4=py36_0
  - defaults/win-64::bottleneck==1.2.1=py36h452e1ab_1
  - defaults/win-64::conda-build==1.21.3=py35_0
  - defaults/win-64::dask==1.1.4=py36_1
  - defaults/win-64::datashape==0.5.4=py36_1
  - defaults/win-64::h5py==2.8.0=py36h3bdd7fb_2
  - defaults/win-64::imageio==2.5.0=py36_0
  - defaults/noarch::keras-applications==1.0.7=py_0
  - defaults/noarch::keras-preprocessing==1.0.9=py_0
  - defaults/win-64::mkl_fft==1.0.6=py36hdbbee80_0
  - defaults/win-64::mkl_random==1.0.1=py36h77b88f5_1
  - defaults/win-64::numba==0.43.1=py36hf9181ef_0
  - defaults/win-64::numexpr==2.6.8=py36h9ef55f4_0
  - defaults/win-64::numpy==1.15.4=py36ha559c80_0
  - defaul



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==custom=py36h363777c_0
  - defaults/win-64::astropy==3.1.2=py36he774522_0
  - https://repo.continuum.io/pkgs/free/win-64::blaze==0.10.1=py35_0
  - defaults/win-64::bokeh==1.0.4=py36_0
  - defaults/win-64::bottleneck==1.2.1=py36h452e1ab_1
  - https://repo.continuum.io/pkgs/free/win-64::conda-build==1.21.3=py35_0
  - defaults/win-64::dask==1.1.4=py36_1
  - defaults/win-64::datashape==0.5.4=py36_1
  - defaults/win-64::h5py==2.8.0=py36h3bdd7fb_2
  - defaults/win-64::imageio==2.5.0=py36_0
  - defaults/noarch::keras-applications==1.0.7=py_0
  - defaults/noarch::keras-preprocessing==1.0.9=py_0
  - defaults/win-64::mkl_fft==1.0.6=py36hdbbee80_0
  - defaults/win-64::mkl_random==1.0.1=py36h77b88f5_1
  - defaults/win-64::numba==0.43.1=py36hf9181ef_0
  - defaults/win-64::numexpr==2.6.8=py36h9ef55f4_0
  - de

DEBUG menuinst_win32:__init__(199): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'C:\Anaconda3', env_name: 'None', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(323): Shortcut cmd is %windir%\System32\WindowsPowerShell\v1.0\powershell.exe, args are ['-ExecutionPolicy', 'ByPass', '-NoExit', '-Command', '"& \'C:\\Anaconda3\\shell\\condabin\\conda-hook.ps1\' ; conda activate \'C:\\Anaconda3\' "']
DEBUG menuinst_win32:__init__(199): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'C:\Anaconda3', env_name: 'None', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(323): Shortcut cmd is C:\Anaconda3\pythonw.exe, args are ['C:\\Anaconda3\\cwp.py', 'C:\\Anaconda3', 'C:\\Anaconda3\\pythonw.exe', 'C:\\Anaconda3\\Scripts\\spyder-script.py']
DEBUG menuinst_win32:create(323): Shortcut cmd is C:\Anaconda3\python.exe, args are ['C:\\Anaconda3\\cwp.py', 'C:\\Anaconda3', 'C:\\Anaconda3\\python.exe', 'C:\\Anaconda3\\Scripts\\spyder-script.py', '--reset']


<h1>
Assignment 2: Segmenting and Clustering Neighborhoods in Toronto
</h1>

<h4>
    <center> Data Science Capstone </center>
</h4>

In [11]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

#for web-scrapping
import requests
import lxml.html as lh

print('Libraries imported.')

Libraries imported.


In [134]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

<h4>
    Reading url using Pandas easily
</h4>

In [142]:
lol = pd.read_html(url)
col=['PostalCode', 'Borough', 'Neighborhood']
lol = pd.DataFrame(lol[0])
lol = lol[~lol.Borough.str.contains('Not assigned')]
lol.reset_index(drop=True, inplace=True)
lol.drop_duplicates()
lol['Neighborhood'].replace("",lol['Borough'], inplace=True)
lol

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [143]:
#data=pd.DataFrame()
data=[]
col=['PostalCode', 'Borough', 'Neighborhood']
for t in tr_elements[1:]:
    name=[]
    for T in t:
        element=T.text_content().rstrip()
        name.append(element)
    data.append(name)

In [144]:
init_df = pd.DataFrame(data[:-4], columns=col)
init_df = init_df[~init_df.Borough.str.contains('Not assigned')]
init_df.reset_index(drop=True, inplace=True)
init_df.drop_duplicates()
init_df['Neighborhood'].replace("",init_df['Borough'], inplace=True)

In [145]:
init_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [146]:
init_df.shape

(103, 3)